# Proyecto precios casas (continuación)
## Segunda parte

In [105]:
import pandas as pd
import seaborn as sns
from plotly import express as px
from matplotlib import pyplot as plt
from matplotlib import gridspec 
from pandas_profiling import ProfileReport
# import geocoder
import time

pd.set_option('display.float_format', lambda x: '%.2f' % x )

In [ ]:
## ETL 
Extract
Transform
Load

### Extraction

In [64]:
data = pd.read_csv('../data/kc_house_data.csv')

In [8]:
print('La base de datos tiene {} filas y {} columnas'.format(data.shape[0],data.shape[1]))

La base de datos tiene 21613 filas y 21 columnas


### Transformation

In [13]:
data.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [65]:
data['date'] = pd.to_datetime(data['date'], format = '%Y-%m-%d')
data['yr_built']= pd.to_datetime(data['yr_built'], format = '%Y').dt.year
data['yr_renovated'] = data['yr_renovated'].apply(lambda x: pd.to_datetime(x, format ='%Y') if x >0 else x )

In [92]:
data['id'] = data['id'].astype(str)
data['zipcode'] = data['zipcode'].astype(str)

In [93]:
data.dtypes

id                       object
date             datetime64[ns]
price                   float64
bedrooms                  int64
bathrooms               float64
sqft_living               int64
sqft_lot                  int64
floors                  float64
waterfront                int64
view                      int64
condition                 int64
grade                     int64
sqft_above                int64
sqft_basement             int64
yr_built                  int64
yr_renovated             object
zipcode                  object
lat                     float64
long                    float64
sqft_living15             int64
sqft_lot15                int64
dtype: object

### Creando nuevas columnas

#### House_age

In [95]:
data.loc[data['yr_built']=>1990,'house_age'] = 'new_house' 
data.loc[data['yr_built']<1990,'house_age'] = 'old_house'

#### Dormitory_type

In [96]:
data.loc[data['bedrooms']<=1, 'dormitory_type'] = 'studio'
data.loc[data['bedrooms']==2, 'dormitory_type'] = 'apartment'
data.loc[data['bedrooms']>2, 'dormitory_type'] = 'house'

#### Condition_type

In [97]:
data.loc[data['condition']<=2, 'condition_type'] = 'bad'
data.loc[data['condition'].isin([3,4]), 'condition_type'] = 'regular'
data.loc[data['condition']== 5, 'condition_type'] = 'good'

#### Price_tier

In [98]:

data['price_tier'] = data['price'].apply(lambda x: 'tier 1' if x <= 321950 else
                                                      'tier 2' if (x > 321950) & (x <= 450000) else
                                                      'tier 3' if (x > 450000) & (x <= 645000) else
                                                      'tier 4')

#### Address, neighbourhood and city (API)

In [ ]:
# for i in range(df_test.shape[0]):
# #     geolocator = Nominatim(user_agent='ErnestoPerez'+str(i))
# #     df_test.loc[i,'lat'], df_test.loc[i,'long']
#     geoloc = geocoder.osm([df_test.loc[i,'lat'], df_test.loc[i,'long']], method='reverse')
#     try:
#         df_test.loc[i,'house_number'] = geoloc.raw['address']['house_number']
#     except:
#         None
#     try:
#         df_test.loc[i,'road'] = geoloc.raw['address']['road']
#     except:
#         None
#     try:
#         df_test.loc[i,'neighbourhood'] = geoloc.raw['address']['neighbourhood']
#     except:
#         None
#     try:
#          df_test.loc[i,'city'] =geoloc.raw['address']['city']
#     except:
#         None
#     time.sleep(2)
#     if i % 150 == 0: 
#         print('Voy en la {}-ésima iteración'.format(i))

# Exploratory Descriptive Analysis (EDA)

In [108]:
data.loc[:5000,:]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,house_age,dormitory_type,condition_type,price_tier
0,7129300520,2014-10-13,221900.00,3,1.00,1180,5650,1.00,0,0,...,0,98178,47.51,-122.26,1340,5650,old_house,house,regular,tier 1
1,6414100192,2014-12-09,538000.00,3,2.25,2570,7242,2.00,0,0,...,1991-01-01 00:00:00,98125,47.72,-122.32,1690,7639,old_house,house,regular,tier 3
2,5631500400,2015-02-25,180000.00,2,1.00,770,10000,1.00,0,0,...,0,98028,47.74,-122.23,2720,8062,old_house,apartment,regular,tier 1
3,2487200875,2014-12-09,604000.00,4,3.00,1960,5000,1.00,0,0,...,0,98136,47.52,-122.39,1360,5000,old_house,house,good,tier 3
4,1954400510,2015-02-18,510000.00,3,2.00,1680,8080,1.00,0,0,...,0,98074,47.62,-122.05,1800,7503,old_house,house,regular,tier 3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,7230400430,2014-09-30,322400.00,3,1.75,1710,15844,1.00,0,0,...,0,98059,47.47,-122.10,1990,20359,old_house,house,regular,tier 2
4997,7140600190,2014-09-05,233500.00,3,1.50,1580,10517,1.00,0,0,...,0,98002,47.29,-122.21,1400,10658,old_house,house,regular,tier 1
4998,6817801410,2014-06-24,400000.00,3,2.00,1230,11413,1.00,0,0,...,0,98074,47.63,-122.03,1570,11517,old_house,house,regular,tier 2
4999,6430500010,2014-06-20,547000.00,5,2.50,2200,4080,1.50,0,0,...,0,98103,47.69,-122.35,1300,4080,old_house,house,good,tier 3


In [123]:
df_test = data.drop(columns =['id','zipcode','yr_renovated'])

In [124]:
import sweetviz as sv

my_report = sv.analyze(df_test.loc[:10,:])
my_report.show_html() 

                                             |          | [  0%]   00:00 -> (? left)

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [126]:
prof = ProfileReport(data.drop(columns =['id','zipcode','yr_renovated']))
prof.to_file(output_file='data-EDA.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Validación de hipótesis

In [118]:
from IPython.display import Image
Image(url= "Precios_Casas.png", width=700, height=700)